# PipeLine ETL

## 1. Conexión a MySQL

Imports

In [1]:
import mysql.connector
import pandas as pd

Conexion

In [2]:
con = mysql.connector.connect(user = "root", password = "1234", host = "127.0.0.1")
cursor = con.cursor()

# 2. Crear Base de Datos y Tablas

In [3]:
cursor.execute("DROP DATABASE f1_db")

In [4]:
cursor.execute("CREATE DATABASE IF NOT EXISTS f1_db")


In [5]:
cursor.execute("USE f1_db")

### Tabla Piloto

In [6]:
sql = """
CREATE TABLE PILOTO (
	driverId INT PRIMARY KEY,
    driverRef VARCHAR(50),
    driverNumber INT,
    driverCode VARCHAR(3),
    forename VARCHAR(50),
    surname VARCHAR(50),
    dob DATE,
    nationality VARCHAR(50),
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

### Tabla Equipo

In [7]:
sql = """
CREATE TABLE EQUIPO(
	constructorId INT PRIMARY KEY,
    constructorRef VARCHAR(50),
    constructorName VARCHAR(70),
    nationality VARCHAR(50),
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

### Tabla Circuito

In [8]:
sql = """
CREATE TABLE CIRCUITO(
	circuitId INT PRIMARY KEY,
    circuitRef VARCHAR(50),
    circuitName VARCHAR(70),
    location VARCHAR(50),
    country VARCHAR(50),
    lat DECIMAL(10, 6),
    lng DECIMAL(10, 6),
    alt INT,
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

### Tabla Carrera

In [9]:
sql = """
CREATE TABLE CARRERA(
	raceId INT PRIMARY KEY,
    raceYear INT,
    raceRound INT,
    circuitId INT,
    FOREIGN KEY (circuitId) REFERENCES CIRCUITO(circuitId),
    raceName VARCHAR(100),
    raceDate DATE,
    raceTime TIME,
    url VARCHAR(255)
    );
"""
cursor.execute(sql)

### Tabla Resultado

In [10]:
sql = """
CREATE TABLE RESULTADO(
	resultId INT PRIMARY KEY,
    raceId INT,
    FOREIGN KEY (raceId) REFERENCES CARRERA(raceId),
    driverId INT,
    FOREIGN KEY (driverId) REFERENCES PILOTO(driverId),
    constructorId INT,
	FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId),
    grid INT,
    positionResult INT,
    points DECIMAL(6,2),
    laps INT,
    totalTime VARCHAR(50)
    );
"""
cursor.execute(sql)

### Tabla clasificación

In [11]:
sql = """
CREATE TABLE CLASIFICACION(
	qualifyingId INT PRIMARY KEY,
	raceId INT,
	FOREIGN KEY (raceId) REFERENCES CARRERA(raceId),
    driverId INT,
    FOREIGN KEY (driverId) REFERENCES PILOTO(driverId),
    constructorId INT,
	FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId),
	qualifyingPosition INT,
    q1 VARCHAR(50),
    q2 VARCHAR(50),
    q3 VARCHAR(50)
    );
""" 
cursor.execute(sql) 

### Tabla info_circuito (scraping)


In [12]:
sql = """
CREATE TABLE INFO_CIRCUITO(
    circuitId INT PRIMARY KEY,  
    FOREIGN KEY (circuitId) REFERENCES CIRCUITO(circuitId),
    capacity VARCHAR(50),
    opened_year INT,            
    length VARCHAR(50),         
    turns INT
);
"""
cursor.execute(sql)

### Tabla info_equipo (scraping)

In [13]:
sql = """
CREATE TABLE INFO_EQUIPO(
	constructorId INT PRIMARY KEY,
    FOREIGN KEY (constructorId) REFERENCES EQUIPO(constructorId),
    championships INT,
    race_wins INT,
    pole_positions INT,
    fastest_laps INT 
    );
"""
cursor.execute(sql)

## 3. Limpieza